In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

In [ ]:
df = pd.read_csv('Ulst6Jan87.csv') #historic gravity data
HistGravData = gpd.GeoDataFrame(df[['id', 'OBS (mgal)']], #Keep only the ID and the reading in milligals.
                            geometry=gpd.points_from_xy(df['LONG'], df['LAT']),crs='epsg:4326') 

In [ ]:
HistGravData.head() #sanity check

In [ ]:
# m = HistGravData.explore('id',marker_type='marker',popup=True,legend=False)

In [ ]:
df = pd.read_csv('OSNIAbsoluteGravity.csv') #Absolute Gravity Data
AbsGravData = gpd.GeoDataFrame(df['Station Name'], #Keep only Station Name
                            geometry=gpd.points_from_xy(df['Easting'], df['Northing']),crs='epsg:2157') #Source data is Irish Transverse Mercator

In [ ]:
AbsGravData.head()

In [ ]:
AbsGravData.to_crs ("EPSG:4326")

In [119]:
#AbsGravData.explore('Station Name',m=m, marker_type='marker', popup=True, legend=False,)
m = AbsGravData.explore('Station Name',m=m, marker_type='marker', popup=True, legend=False,)

In [116]:
Ab_buffer = AbsGravData.buffer(5)

In [ ]:
Ab_buffer.explore(m=m)